### Basic aperture photometry using weirdly-shaped apertures, using ds9 regions files, and pyregion to filter the images by those regions

This adapts my pyregion tutorial, to do simple photometry of HST for SGAS J0033.

In [1]:
import numpy as np
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.io.fits import getdata
import pyds9
import pyregion

In [2]:
# Set up locations
imdir = "/Users/jrrigby1/Dropbox/S0033/HST/"
imfile1 = imdir + "final_images/SGAS0033_F410M_full_0.03g0.5_drc_sci.fits"
imfile2 = imdir + "final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits"
reg = 'image;box(2465.6667,1928.6667,17.51326,92.948953,20)'
bkg = 'image;box(2365.6667,1928.6667,17.51326,200.948953,20)'

f = fits.open(imfile1)
dum = f[0].data  # screwing around to get hdu as pyregions wants it
im = dum.astype(np.float64)
f2 = fits.open(imfile2)
dum2 = f2[0].data  # screwing around to get hdu as pyregions wants it
im2 = dum2.astype(np.float64)

#(im_int, hdr) = getdata(image_file, header=True) #image is numpy array
#d = pyds9.DS9('foo1')  # start ds9.  'd' is the way to call ds9
d = pyds9.DS9('/var/folders/ly/n6803yys5cs9tw41zytqgmvcmz2qhx/T//DS9_foo1.39834')
d.set_np2arr(im) # sending ndarray im directly to ds9
d.set("colorbar no")   # example of manipulating the ds9 window
d.set("scale zscale")  # example of manipulating the ds9 window
d.set("zoom to 1.5 1.5")
d.set("pan to 2460 1920")

mreg = pyregion.open(imdir + 'mike_phot555.reg').as_imagecoord(f[0].header)
mbkg = pyregion.open(imdir + 'mike_phot555_bkg.reg').as_imagecoord(f[0].header)
r = pyregion.parse(reg)
b = pyregion.parse(bkg)
d.set('regions', reg)
d.set('regions', bkg)

1

In [20]:
def pyregions_photometry(object_region, sky_region, im) :  # This was workaround for old pyreg
    # Pyregions is barfing on simpler method
    myfilter   = object_region.get_filter()
    bkgfilter  = sky_region.get_filter()
    total_flux = np.nansum(myfilter.mask(im) * im)  #nansum treats nans as zeros for summation
    npix_reg   = np.count_nonzero(myfilter.mask(im))
    npix_bkg   = np.count_nonzero(bkgfilter.mask(im))
    bkg_med    = np.nanmean(bkgfilter.mask(im) * im)
    bkg_flux   = bkg_med * npix_reg / npix_bkg
    net_flux   = total_flux - bkg_flux
    print "Npix in regions", npix_reg, npix_bkg
    return(total_flux, bkg_flux, net_flux)

def pyregions_photometry2(regfile, bkgregfile, imagefile):
    f = fits.open(imagefile)
    dum = f[0].data  # screwing around to get hdu as pyregions wants it
    im = dum.astype(np.float64)
    reg = pyregion.open(regfile).as_imagecoord(f[0].header)
    bkg = pyregion.open(bkgregfile).as_imagecoord(f[0].header)
    total_flux = np.nansum(mreg.get_mask(f[0]) * im)
    bkg_flux   = np.nansum(mbkg.get_mask(f[0]) * im)
    return(total_flux, bkg_flux)

In [22]:
print pyregions_photometry2(imdir + 'mike_phot555.reg', imdir + 'mike_phot555_bkg.reg', imfile1)

#r = pyregion.parse(reg)
total_flux = np.nansum(mreg.get_mask(f[0]) * im)
#b = pyregion.parse(bkg)
bkg_flux   = np.nansum(mbkg.get_mask(f[0]) * im)
print total_flux, bkg_flux

(6.5199348444139105, 0.27350317177521788)
6.51993484441 0.273503171775


In [9]:
print "Simple aperture photometry:"
print "total_flux  flux_in_bkg   net_flux"
print pyregions_photometry(r, b, im)
print pyregions_photometry(r, b, im2)
for ii in range(0,len(mreg)-1,1) :
    print pyregions_photometry(mreg[ii], mbkg, im)

Simple aperture photometry:
total_flux  flux_in_bkg   net_flux
Npix in regions 1628 3518
(5.6707804760312683, 5.0773905650809883e-09, 5.6707804709538774)
Npix in regions 1628 3518
(35.64421488676453, 7.035078271095579e-08, 35.644214816413751)


AttributeError: 'Shape' object has no attribute 'get_filter'

In [ ]:
# In morning, derive constant to multiply cts in F555W, to subtract F410 cont...
